In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
print (model.names)

In [ ]:
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import cv2

# Load your model -trained on 3 classes
MODEL= "/content/drive/MyDrive/content/runs/detect/train/weights/best.pt"
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

classes_to_count = [0,1,2]

# Setup video capture

cap = cv2.VideoCapture("/content/drive/MyDrive/DJI_0495.MP4")
assert cap.isOpened(), "Error opening video stream or file"

# Define your line points for counting
line_points = [(1126, 1118), (2786, 1118)]  # Adjust these points as needed

# Initialize Object Counter with desired settings
counter = object_counter.ObjectCounter()
counter.set_args(view_img=True, reg_pts=line_points, classes_names=model.names)

# Define the output video path and parameters
output_video_path = "/content/drive/MyDrive/output_video.mp4"
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_video = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    tracks = model.track(frame, persist=True, show=False, classes=classes_to_count)
    frame = counter.start_counting(frame, tracks)

    # Write the frame to the output video
    out_video.write(frame)

# Release the VideoWriter
out_video.release()

for class_id in classes_to_count:
    in_count = counter.in_counts if isinstance(counter.in_counts, int) else counter.in_counts.get(class_id, 0)
    out_count = counter.out_counts if isinstance(counter.out_counts, int) else counter.out_counts.get(class_id, 0)

    print(f"In count for {model.names[class_id]}: {in_count}")
    print(f"Out count for {model.names[class_id]}: {out_count}")


cap.release()
cv2.destroyAllWindows()